### COSC 89.34: AI Agents (PS #0)
Ali Azam

#### Part I
1. (b) All three
2. (b) Bias decreases, variance increases
3. (b) A is positive semi-definite
4. (a) Only on current state _s_ and action _a_
5. (c) _f_ is strongly convex and has Lipschitz-continuous gradients
6. (a) Distribution shift
7. (c) $O(n^2)$
8. (a) Network's final linear can achieve zero empirical error
9. (b) Requires summing over all possible hypotheses
10. (a) Noise inherent in data-generation process
11. (b) SIMD
12. (c) Design rules s.t. self-interested agents produce optinal outcome for designer
13. (d) Pure strategy Nash Equilibria
14. (c) If the new feature had no effect, there wouold be a 3% chance of observing an increase at least this large by chance
15. (b) Bet (since pass = 5 vs. bet = $0.2 \cdot 100 + 0.8 \cdot -10 = 12$)
16. (c) $\frac{1}{1-0.9}=10.0$
17. (b) UDP
18. (a) Maximize ELBO $L(\lambda)$
19. (b) $Q$, $K$, $V$ are linear projections of the residual streams
20. (d) Inference FLOPs per token _decrease_ as a power law model parameter count increases


#### Part II
1. By discounted return,
$$
V = \sum_{k=0}^{\infty} \gamma^k r = r\sum_{k=0}^{\infty} \gamma^k = r \cdot \frac{1}{1-\gamma}
\newline
= 5 \cdot \frac{1}{1-0.8}  \text{ since $\gamma$=0.8 and reward=5}
\newline
= 5 \cdot 5 = 25
$$

2. Since $V_k(s_A) = 10$, $V_k(s_B) = 20$, $\gamma = 0.9$, we have the outcomes:
$\newline$
$s_A$ (with probability = 0.5), reward 0: return $0 + 0.9 \cdot 10 = 9$
$\newline$
$s_B$ (with probability = 0.5), reward 10: return 10 + 0.9 \cdot 20 = 28$
$\newline$
Using the formula for expctation:
$\newline V_k+1(s_A) = 0.5 \cdot 9 + 0.5 \cdot 28 = 18.5$

3. Given $z_i = \sum_{j=1}^{n} \alpha_{ij} v_j$, and $v_j = W_Vx_j$, then for token 1:
$\newline$
$z_1 = \alpha_{11}v_1 + \alpha{12}v_2 = \alpha_{11}W_Vx_1 + \alpha_{12}W_Vx_2$
$\newline$
Residual add:
$\newline$
$x_1' = x_1 + z_1 = x_1 + \alpha_{11}v_1 + \alpha{12}v_2 = \alpha_{11}W_Vx_1 + \alpha_{12}W_Vx_2$

4. Not entirely sure about this one, but I'd expect something to do with sending tensors to the GPU in `inputs.to("cuda")` from the CPU, e.g. loading the data or preprocessing.

5. If that is the case, use a Straight-Through Estimator as follows: for the forward pass use the round/quantize `round(x)`, but  in the packward treat quantization as identity ($\frac{\partial \texttt{round(x)}}{\partial x})$ which should approximately equal 1 when inside the range. This can also be clipped for different usages.

#### Part III

1. (a) and (b)

In [ ]:
# !pip install torch==2.7.1 torchvision==0.22.1 scikit-learn==1.7.2 numpy==2.1.3

import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


SEED = 8934
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


class RCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2) # 28x28 -> 14x14
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2  = nn.MaxPool2d(2, 2) # 14x14 -> 7x7
        
        self.fc = nn.Linear(32 * 7 * 7, 128)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            f = model(x)
            features.append(f.cpu().numpy())
            labels.append(y.numpy())
    return np.vstack(features), np.concatenate(labels)


transform = transforms.ToTensor()
train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=256, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=256, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RCNN().to(device)
model.eval()


train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

svm = LinearSVC(C=1.0, random_state=SEED, max_iter=5000)
svm.fit(train_features, train_labels)

y_pred = svm.predict(test_features)
acc = accuracy_score(test_labels, y_pred)

print(f'Test Accuracy: {acc * 100:.2f}%')

Test Accuracy: 94.88%


(c) 

(i) A randomly initialized, untrained CNN is able to produce highly informative features fro the MNIST dataset, and the linear classifier on top of that achieves high accuracy without learning any convolutional weights. This is unusually high, but shows that that the CNN's dimensional features had well-developed strutures for identification between different classes.

(ii) The CNN architecture is very helpful since the convolutions give a good sense of locality and object detection within the large dataset with small shifts. ReLU introduces nonlinearity into the system (and is more nontrivial than other features of the architecture), and pooling across the layers gives a good sense of invariant features for the same digit classes, for example, across the large dataset. Stacking all of these gives random nonlinear projections that perform very well in separating the MNIST classes. The success of the model relies more so on the architectural properties than on learned weights.

(iii) Since the kernel already performs so well without supervised training, the CNN is already aligned with MNIST at initialization. The action of neural networks (such as CNN's convolutional kernels on the images initialized randomly) then training a linear classifier implicitly makes a kernel classification for it. This is supported by Rahimi and Recht, who show that nonlinear mappings by these random features followed by linear models can approximate kernel machines, since these features allow the data to have a high degree of linear separability (e.g. allowing us to train an SVC on the MNIST dataset).

2. Note: output exported from this code run on [Colab notebook](https://colab.research.google.com/drive/1Wz4j5giLM3tLikTdloOub2hUq22S3NGe?usp=sharing) with T4 GPU

(b)

In [ ]:
# !pip install torch==2.7.1 matplotlib==3.10.3 numpy==2.1.3
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O shakespeare.txt

DATA_PATH = './shakespeare.txt'

with open(DATA_PATH, 'r', encoding='utf-8') as f:
    text = f.read()


import math
import random

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

 
SEED = 8934
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


BLOCK_SIZE = 128
BATCH_SIZE = 64

d_model = 384
n_heads = 6
n_layers = 6
d_ff = 4 * d_model
dropout = 0.2
lr = 3e-4
max_iters = 5000
eval_interval = 500
eval_iters = 200

chars = sorted(list(set(text)))
n_vocab = len(chars)
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [char_to_idx[c] for c in s]          # encode string to list of integers
decode = lambda l: ''.join([idx_to_char[i] for i in l]) # decode list of integers to string

n = int(0.9 * len(text))
# Convert to tensors
train_data = torch.tensor(encode(text[:n]), dtype=torch.long)
val_data = torch.tensor(encode(text[n:]), dtype=torch.long)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_batch(split: str, device: str):
    """Helper method to generate small batch of inputs x and targets y."""
    
    d = train_data if split == "train" else val_data
   
    # random starting positions
    ix = torch.randint(0, len(d) - BLOCK_SIZE - 1, (BATCH_SIZE,))
    x = torch.stack([d[i : i + BLOCK_SIZE] for i in ix])
    y = torch.stack([d[i + 1 : i + BLOCK_SIZE + 1] for i in ix])
    return x.to(device), y.to(device)


@torch.no_grad()
def estimate_loss(model: nn.Module, device: str):
    model.eval()
    out = {}
    for split in ["train", "val"]:
        losses = []
        for _ in range(eval_iters):
            x, y = get_batch(split, device)
            _, loss = model(x, y)
            losses.append(loss.item())
        out[split] = float(np.mean(losses))
    model.train()
    return out

In [ ]:
# Components
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dropout: float):
        super().__init__()
        assert d_model % n_heads == 0
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        self.q_proj = nn.Linear(d_model, d_model, bias=False)
        self.k_proj = nn.Linear(d_model, d_model, bias=False)
        self.v_proj = nn.Linear(d_model, d_model, bias=False)
        self.out_proj = nn.Linear(d_model, d_model, bias=False)

        self.attn_drop = nn.Dropout(dropout)
        self.resid_drop = nn.Dropout(dropout)

        # Causal mask, [1, 1, T, T] broadcastable
        mask = torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE, dtype=torch.bool))
        self.register_buffer("causal_mask", mask.view(1, 1, BLOCK_SIZE, BLOCK_SIZE), persistent=False)

    def forward(self, x: torch.Tensor):
        # x: [B, T, C]
        B, T, C = x.shape

        q = self.q_proj(x)  # [B, T, C]
        k = self.k_proj(x)
        v = self.v_proj(x)

        # split heads: [B, nh, T, hd]
        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        # scaled dot-product attention
        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # [B, nh, T, T]

        # apply causal mask for current T
        mask = self.causal_mask[:, :, :T, :T]
        att = att.masked_fill(~mask, float("-inf"))

        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)

        y = att @ v  # [B, nh, T, hd]
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # [B, T, C]

        y = self.out_proj(y)
        y = self.resid_drop(y)
        return y


class FeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class TransformerBlock(nn.Module):
    # Pre-LN block: x + Attn(LN(x)), x + FFN(LN(x))
    def __init__(self, d_model: int, n_heads: int, d_ff: int, dropout: float):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff, dropout)

    def forward(self, x: torch.Tensor):
        x = x + self.attn(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x


class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = BLOCK_SIZE  # Store block_size as instance variable

        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(BLOCK_SIZE, d_model)
        self.drop = nn.Dropout(dropout)

        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        self.ln_f = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

        # weight tying is optional; keep simple and explicit
        # self.lm_head.weight = self.tok_emb.weight

    def forward(self, idx: torch.Tensor, targets: torch.Tensor | None = None):
        # idx: [B, T]
        B, T = idx.shape
        assert T <= self.block_size

        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)  # [1, T]
        x = self.tok_emb(idx) + self.pos_emb(pos)  # [B, T, C]
        x = self.drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.ln_f(x)
        logits = self.lm_head(x)  # [B, T, vocab]

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, self.vocab_size), targets.view(-1))

        return logits, loss

    @torch.no_grad()
    def generate(self, idx: torch.Tensor, max_new_tokens: int, temperature: float = 1.0, top_k: int | None = None):
        self.eval()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]  # crop
            logits, _ = self(idx_cond, targets=None)
            logits = logits[:, -1, :]  # last step, [B, vocab]

            logits = logits / max(temperature, 1e-6)

            if top_k is not None and top_k > 0:
                v, _ = torch.topk(logits, k=min(top_k, logits.size(-1)))
                kth = v[:, -1].unsqueeze(-1)
                logits = torch.where(logits < kth, torch.full_like(logits, float("-inf")), logits)

            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)  # [B, 1]
            idx = torch.cat([idx, next_id], dim=1)
        return idx



In [ ]:
model = DecoderOnlyTransformer(n_vocab).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)


for i in range(1, max_iters + 1):
    xb, yb = get_batch("train", device)
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

    if i % eval_interval == 0 or i == 1:
        losses = estimate_loss(model, device)
        print(f"{i} | train loss: {losses['train']:.4f} | val loss: {losses['val']:.4f}")

In [ ]:
sample_max_new_tokens = 500

prompt = "JULIET:"
x0 = torch.tensor(encode(prompt), dtype=torch.long).unsqueeze(0).to(device)
out = model.generate(
    x0,
    max_new_tokens=500,
    temperature=0.85,
    top_k=50,
)

juliet = decode(out[0].tolist())
print("\n--- SAMPLE ---\n")
print(juliet)
print("\n--- END SAMPLE ---\n")

**Output:**
```
1 | train loss: 3.7021 | val loss: 3.7315
500 | train loss: 1.8726 | val loss: 1.9846
1000 | train loss: 1.5964 | val loss: 1.7753
1500 | train loss: 1.4704 | val loss: 1.6658
2000 | train loss: 1.3979 | val loss: 1.6125
2500 | train loss: 1.3475 | val loss: 1.5668
3000 | train loss: 1.3119 | val loss: 1.5441
3500 | train loss: 1.2763 | val loss: 1.5168
4000 | train loss: 1.2531 | val loss: 1.5139
4500 | train loss: 1.2235 | val loss: 1.5008
5000 | train loss: 1.2027 | val loss: 1.4926

--- SAMPLE ---

JULIET:
God knows he hath not the Capitol' set:
And why, sir? then, pity will I have.

CLARENCE:
But these Earl of Norfolk, The senators we was
so deadly tender heaven the cause throats him approach:
His world, of my dispatch,
So far ore grace as the potion doth the royal duke,
Whose seven branch and give of life. When she was speak, but her,
And thou hast let a party but complime monarches
That they shall for my sake a care for ever
Of her action can pash the devil of wild:
The king of Went, that quee

--- END SAMPLE ---
```

3. Note: output exported from this code run on [Colab notebook](https://colab.research.google.com/drive/1Wz4j5giLM3tLikTdloOub2hUq22S3NGe?usp=sharing) with T4 GPU 

(c)

In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O shakespeare.txt

from collections import Counter

DATA_PATH = './shakespeare.txt'
N_MERGES = 1000

with open(DATA_PATH, 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
def count_pairs(tokens):
    pairs = Counter()
    if len(tokens) < 2:
        return pairs
    for i in range(len(tokens) - 1):
        pairs[(tokens[i], tokens[i + 1])] += 1
    return pairs


def merge_pair(tokens, pair, merged):
    a, b = pair
    out = []
    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1 and tokens[i] == a and tokens[i + 1] == b:
            out.append(merged)
            i += 2
        else:
            out.append(tokens[i])
            i += 1
    return out


def train_bpe(text, num_merges):
    tokens = list(text)
    vocab = set(tokens)
    merges = []
    merge_ranks = {}
    
    for _ in range(num_merges):
        pairs = count_pairs(tokens)
        if not pairs:
            break
        
        best_pair, count = pairs.most_common(1)[0]
        if count < 2:
            break
        
        merged = best_pair[0] + best_pair[1]
        tokens = merge_pair(tokens, best_pair, merged)
        
        merges.append((best_pair, merged))
        merge_ranks[best_pair] = len(merges) - 1
        vocab.add(merged)
    
    return merges, merge_ranks, vocab


def encode(s, merge_ranks):
    """Encode string using learned merge rules."""
    tokens = list(s)
    
    if len(tokens) < 2 or not merge_ranks:
        return tokens
    
    while True:
        pairs = [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]
        candidates = [(merge_ranks[p], p) for p in pairs if p in merge_ranks]
        
        if not candidates:
            break
        
        _, best_pair = min(candidates)
        merged = best_pair[0] + best_pair[1]
        tokens = merge_pair(tokens, best_pair, merged)
        
        if len(tokens) < 2:
            break
    
    return tokens

In [ ]:
merges, merge_ranks, vocab = train_bpe(text, N_MERGES)

print(f"Learned merges: {len(merges)}")

# Show merge rules
show = merges[-50:]
print(f"\n--- MERGE RULES (LAST 50) ---")

# (a)
start = len(merges) - len(show)
for i, (pair, merged) in enumerate(show, start=start):
    print(f"{i:04d}: {pair[0]!r} + {pair[1]!r} -> {merged!r}")

# (b)
horatio = "Alas, poor Yorick! I knew him, Horatio:"
tokens = encode(horatio, merge_ranks)

print("\n--- ENCODING EXAMPLE ---")
print("Input:", repr(horatio))
print("Tokens:", tokens)
print("Token count:", len(tokens))
print("Reconstructed:", repr("".join(tokens)))

**Output:**

```
Learned merges: 1000

--- MERGE RULES (LAST 50) ---
0950: ' d' + 'o ' -> ' do '
0951: 'th' + 'ese ' -> 'these '
0952: 'ee' + 'ch' -> 'eech'
0953: 'H' + 'en' -> 'Hen'
0954: ' m' + 'e, ' -> ' me, '
0955: 'e' + ':\n' -> 'e:\n'
0956: '.\n\nP' + 'ETRUCHIO:\n' -> '.\n\nPETRUCHIO:\n'
0957: ' to ' + 'the ' -> ' to the '
0958: 'you' + ' m' -> 'you m'
0959: 'S' + 't' -> 'St'
0960: 'a' + 'il' -> 'ail'
0961: 'c' + 'rown' -> 'crown'
0962: 'ou' + 't' -> 'out'
0963: ' a' + 'll ' -> ' all '
0964: ' w' + 'as ' -> ' was '
0965: 'QUEEN ' + 'EL' -> 'QUEEN EL'
0966: 'QUEEN EL' + 'I' -> 'QUEEN ELI'
0967: 'QUEEN ELI' + 'Z' -> 'QUEEN ELIZ'
0968: 'QUEEN ELIZ' + 'AB' -> 'QUEEN ELIZAB'
0969: 'QUEEN ELIZAB' + 'ET' -> 'QUEEN ELIZABET'
0970: 'QUEEN ELIZABET' + 'H' -> 'QUEEN ELIZABETH'
0971: 'n' + 'at' -> 'nat'
0972: 't' + 'ongu' -> 'tongu'
0973: 'w' + 'ard' -> 'ward'
0974: 'e' + ';\n' -> 'e;\n'
0975: 'or' + 'row' -> 'orrow'
0976: 'in' + 't' -> 'int'
0977: 'thou' + 'gh' -> 'though'
0978: 'e' + 'e ' -> 'ee '
0979: '?\n\n' + 'B' -> '?\n\nB'
0980: 're' + 'e ' -> 'ree '
0981: 'us' + 'b' -> 'usb'
0982: ';' + ' but ' -> '; but '
0983: 'a' + ' w' -> 'a w'
0984: 'w' + 'ay ' -> 'way '
0985: '.\n\nM' + 'ENENIUS:\n' -> '.\n\nMENENIUS:\n'
0986: 'c' + 'ar' -> 'car'
0987: 'ell' + 'ow' -> 'ellow'
0988: 'by ' + 'the ' -> 'by the '
0989: 'p' + 'a' -> 'pa'
0990: '?\n\n' + 'P' -> '?\n\nP'
0991: 'la' + 'ck' -> 'lack'
0992: 's' + 'ay' -> 'say'
0993: 'g' + 'reat ' -> 'great '
0994: ' g' + 'ood ' -> ' good '
0995: 'is ' + 'the ' -> 'is the '
0996: 'S' + ' ' -> 'S '
0997: 'to ' + 'my ' -> 'to my '
0998: 'S' + 'erv' -> 'Serv'
0999: 'hea' + 'ven' -> 'heaven'

--- ENCODING EXAMPLE ---
Input: 'Alas, poor Yorick! I knew him, Horatio:'
Tokens: ['A', 'la', 's, ', 'po', 'or ', 'Y', 'or', 'ick', '! ', 'I ', 'k', 'new', ' him', ', ', 'H', 'or', 'at', 'io', ':']
Token count: 19
Reconstructed: 'Alas, poor Yorick! I knew him, Horatio:'
```